In [69]:
import pandas as pd 
import numpy as np 
import pickle 
import glob

from itertools import groupby
from operator import itemgetter

In [90]:
def combine_consecutive_values(df, orient = "+"):
    df = df.sort_values(by = "g_pos", ascending = True)
    data = df["g_pos"].values
    block_tags = []
    counter = 1
    v = []
    for k, g in groupby(enumerate(data), lambda ix : ix[0] - ix[1]):
        val_lst = list(map(itemgetter(1), g))
        block_tags = block_tags + [counter] *len(val_lst)
        v.append([np.min(val_lst), np.max(val_lst)])
        counter+=1
    df["block_tags"] = block_tags
    return df



            

In [91]:
ATM_hr = [[5228, 7997]]
BRCA1_hr = [[0,441], [5365, 5589]]
BRCA2_hr = [[1799, 2623], [3504, 4590], [7879,8084], [8243, 8524],[8851,10254]]
CHEK2_hr = [[470, 557], [715,1183]]
MSH6_hr = [[1445,1754], [2117,2342], [3656,4080]]
PALB2_hr = [[0,108], [2559,2674]]
TP53_hr = [[338,389], [659,762],[775,827], [887,1179]]

gene_to_hr = {
    "ATM": ATM_hr,
    "BRCA1":BRCA1_hr,
    "BRCA2":BRCA2_hr,
    "CHEK2":CHEK2_hr,
    "MSH6":MSH6_hr,
    "PALB2":PALB2_hr,
    "TP53": TP53_hr
}

In [104]:
ATM_lr = [[0, 1547], [3284, 3873], [7997, 8293], [8737,9168]]
BRCA1_lr = [[2596, 3179], [4327, 4484], [4992,5258]]
BRCA2_lr = [[229, 831], [4590, 5961]]
CHEK2_lr = [[254,470], [1183,1283]]
MSH6_lr = [[431, 1346], [1754,2117], [2927, 3227]]
PALB2_lr = [[2674,3558]]
TP53_lr = [[105,338],[389,524],[596,659], [726,775], [827,887]]

gene_to_lr = {
    "ATM": ATM_lr,
    "BRCA1":BRCA1_lr,
    "BRCA2":BRCA2_lr,
    "CHEK2":CHEK2_lr,
    "MSH6":MSH6_lr,
    "PALB2":PALB2_lr,
    "TP53": TP53_lr
}

In [103]:
bed_lines = []

for gene, hr_lst in gene_to_hr.items():
    files = glob.glob(f"{gene}_exon_*.pickle")
    vals = []
    for fn in files:
        with open(fn, "rb") as f:
            data = pickle.load(f)
        f.close()
        for d in data:
            if "canonical" in d.keys():
                vals.append([d["cdna_start"], d["genomic_pos"], d["chrom"]])
        
    df = pd.DataFrame(vals, columns = ["cdna", "g_pos", "chr"])
    for (start, end) in hr_lst:
        subset = df.loc[(df["cdna"] > start) & (df["cdna"] <= end)]
        df_a = combine_consecutive_values(subset)

        contained = set(range(start + 1, end + 1))
        assert len(contained) == len(set(df_a["cdna"].values))
        
        block_ids = list(set(df_a["block_tags"]))
        sorted(block_ids)
        chrom = df_a.iloc[0]["chr"]
        for block in block_ids:
            s1 = df_a.loc[df_a["block_tags"] == block]
            start_val = np.min(s1["g_pos"])
            #index 1 more on the upper end 
            end_val = np.max(s1["g_pos"]) + 1
            bed_lines.append(["chr" + str(chrom) , start_val, end_val, gene + "_Higher_Risk_Region"])
        
        
bed_df = pd.DataFrame(bed_lines, columns = ["chrom", "chromStart", "chromEnd", "name"])
bed_df.to_csv("higher_risk_regions.bed", index = False, header = False, sep = "\t")

In [105]:
bed_lines = []

for gene, lr_lst in gene_to_lr.items():
    files = glob.glob(f"{gene}_exon_*.pickle")
    vals = []
    for fn in files:
        with open(fn, "rb") as f:
            data = pickle.load(f)
        f.close()
        for d in data:
            if "canonical" in d.keys():
                vals.append([d["cdna_start"], d["genomic_pos"], d["chrom"]])
        
    df = pd.DataFrame(vals, columns = ["cdna", "g_pos", "chr"])
    for (start, end) in lr_lst:
        subset = df.loc[(df["cdna"] > start) & (df["cdna"] <= end)]
        df_a = combine_consecutive_values(subset)

        contained = set(range(start + 1, end + 1))
        assert len(contained) == len(set(df_a["cdna"].values))
        
        block_ids = list(set(df_a["block_tags"]))
        sorted(block_ids)
        chrom = df_a.iloc[0]["chr"]
        for block in block_ids:
            s1 = df_a.loc[df_a["block_tags"] == block]
            start_val = np.min(s1["g_pos"])
            #index 1 more on the upper end 
            end_val = np.max(s1["g_pos"]) + 1
            bed_lines.append(["chr" + str(chrom) , start_val, end_val, gene + "_Lower_Risk_Region"])
        
        
bed_df = pd.DataFrame(bed_lines, columns = ["chrom", "chromStart", "chromEnd", "name"])
bed_df.to_csv("lower_risk_regions.bed", index = False, header = False, sep = "\t")

,cdna,g_pos,chr,block_tags
527,441,43104235,17,1
528,440,43104236,17,1
529,439,43104237,17,1
530,438,43104238,17,1
531,437,43104239,17,1
...,...,...,...,...
89,5,43125360,17,6
90,4,43125361,17,6
91,3,43125362,17,6
92,2,43125363,17,6


In [54]:
v

[[1, 1], [4, 6], [10, 10], [15, 18], [22, 22], [25, 28]]